In [1]:
import pandas as pd
import numpy as np

Import the dataset

In [2]:
file_path = 'dataset_ecommerce.csv'

In [3]:
df = pd.read_csv('dataset_ecommerce.csv')

Initial Data Inspection

First 5 Rows of the raw data

In [4]:
print(df.head(10))

   order_id  product_id  order_date courier_delivery        city  \
0    738527         194  2022-12-12              JNE   Palembang   
1    149822         268  2023-04-13          SiCepat  Yogyakarta   
2    635991         175  2023-01-01              JNE    Surabaya   
3    221362         257  2023-05-15    POS Indonesia      Bekasi   
4    687298         269  2022-07-30              JNE     Bandung   
5    599035         191  2022-07-03              JNE  Yogyakarta   
6    818755         112  2023-01-12              JNE    Surabaya   
7    564488         257  2022-12-04              JNE    Makassar   
8    633998         228  2023-05-01          SiCepat       Depok   
9    578132         198  2022-11-02     Ninja Xpress   Pekanbaru   

           district type_of_delivery  estimated_delivery_time_days  \
0       Bukit Kecil          Reguler                           4.0   
1            Terban         Same Day                           5.0   
2        Sambikerep         Same Day     

Data Information

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 507111 entries, 0 to 507110
Data columns (total 10 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   order_id                      507111 non-null  int64  
 1   product_id                    507111 non-null  int64  
 2   order_date                    507111 non-null  object 
 3   courier_delivery              507110 non-null  object 
 4   city                          507110 non-null  object 
 5   district                      507110 non-null  object 
 6   type_of_delivery              507110 non-null  object 
 7   estimated_delivery_time_days  507110 non-null  float64
 8   product_rating                507110 non-null  float64
 9   ontime                        507110 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 38.7+ MB


Descriptive Statistics for numeric columns

In [6]:
print(df.describe())

            order_id     product_id  estimated_delivery_time_days  \
count  507111.000000  507111.000000                 507110.000000   
mean   500327.976064     205.421476                      3.000071   
std    288482.333665      54.864231                      1.413442   
min         0.000000     111.000000                      1.000000   
25%    250856.000000     158.000000                      2.000000   
50%    500443.000000     205.000000                      3.000000   
75%    750302.000000     253.000000                      4.000000   
max    999998.000000     300.000000                      5.000000   

       product_rating  
count   507110.000000  
mean         3.000483  
std          1.414362  
min          1.000000  
25%          2.000000  
50%          3.000000  
75%          4.000000  
max          5.000000  


Check for missing values

In [7]:
print(df.isnull().sum())

order_id                        0
product_id                      0
order_date                      0
courier_delivery                1
city                            1
district                        1
type_of_delivery                1
estimated_delivery_time_days    1
product_rating                  1
ontime                          1
dtype: int64


Data Cleaning And Type Conversion

Rename the column for consistency

In [8]:
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace(':', '').str.replace('/', '_').str.replace('-', '_').str.strip()

In [9]:
print(df.columns.tolist())

['order_id', 'product_id', 'order_date', 'courier_delivery', 'city', 'district', 'type_of_delivery', 'estimated_delivery_time_days', 'product_rating', 'ontime']


Convert Order_date column to datetime objects

In [10]:
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')

In [11]:
df.dropna(subset=['order_date'], inplace=True)
print(f"\n--- Rows after handling missing or invalid order_dates: {len(df)} ---")



--- Rows after handling missing or invalid order_dates: 507111 ---


Convert other relevant columns to appropriate numeric types

In [12]:
numeric_cols_to_clean = [
    'unit_price', 'shipping_cost', 'product_rating',
    'estimated_delivery_time_days', 'customer_care_calls', 'prior_purchases'
    # Add any other columns you see that should be numbers, like 'discount_offered' or 'weight_in_gms'
]

In [13]:
for col in numeric_cols_to_clean:
    if col in df.columns: # Check if column actually exists in DataFrame
        # If the column's current type is 'object' (text), it might contain non-numeric characters.
        if df[col].dtype == 'object':
            # Remove common non-numeric characters like '$', ',', '%' from strings before converting to number
            df[col] = df[col].astype(str).str.replace(r'[$,%]', '', regex=True)
        # Convert to numeric. errors='coerce' turns unconvertible values into NaN.
        df[col] = pd.to_numeric(df[col], errors='coerce')


Handles remaining missing values (NaNs) in numeric columns

In [14]:
for col in ['unit_price', 'shipping_cost', 'customer_care_calls', 'prior_purchases']:
    if col in df.columns:
        if df[col].isnull().any(): # Only attempt to fill if there are actual NaNs
            df[col].fillna(0, inplace=True) # Assuming 0 for missing quantities/costs/counts

for col in ['product_rating', 'estimated_delivery_time_days']:
    if col in df.columns:
        if df[col].isnull().any():
            df[col].fillna(df[col].median(), inplace=True) # Median for continuous metrics like ratings/times


/tmp/ipython-input-14-1181154811.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True) # Median for continuous metrics like ratings/times


Drop Dupilcates Rows

In [15]:
df.drop_duplicates(inplace=True)

In [16]:
print(f"--- Rows after dropping duplicates: {len(df)} ---")

--- Rows after dropping duplicates: 507111 ---


Final check of data types and null

In [17]:
print("\n--- Final Data Types after cleaning ---")
print(df.dtypes)
print("\n--- Missing Values after cleaning ---")
print(df.isnull().sum())


--- Final Data Types after cleaning ---
order_id                                 int64
product_id                               int64
order_date                      datetime64[ns]
courier_delivery                        object
city                                    object
district                                object
type_of_delivery                        object
estimated_delivery_time_days           float64
product_rating                         float64
ontime                                  object
dtype: object

--- Missing Values after cleaning ---
order_id                        0
product_id                      0
order_date                      0
courier_delivery                1
city                            1
district                        1
type_of_delivery                1
estimated_delivery_time_days    0
product_rating                  0
ontime                          1
dtype: int64


Feature Engineering ( Creating Anomaly Flags )


Sort Date by order_date for accurate rolling calcutions

In [18]:
df = df.sort_values(by='order_date').reset_index(drop=True)

Anamoly Area 1 : Shipping Performance Anomalies

Flag A: Unusaully long estimated delivery time (Anomaly if estimated time is in the top 5% of all estimated times)

In [19]:
long_estimated_delivery_threshold = 4 # Explicitly set threshold to capture estimated times of 5 days
df['long_estimated_delivery_anomaly_flag'] = np.where(
    df['estimated_delivery_time_days'] > long_estimated_delivery_threshold,
    'LONG_ESTIMATED_DELIVERY',
    None
)
print(f"--- Number of 'LONG_ESTIMATED_DELIVERY' flags detected: {df['long_estimated_delivery_anomaly_flag'].count()} ---")
# Why: We've adjusted the threshold based on data distribution to ensure anomalies are captured.

--- Number of 'LONG_ESTIMATED_DELIVERY' flags detected: 101198 ---


Flag B: Not On Time Delivery (Directly from 'ontime' column)

In [20]:
# Flag B: Not On Time Delivery (DISABLED for this dataset)
# Since df['ontime'].value_counts() shows only 1, this flag cannot be used.
# df['not_ontime_delivery_anomaly_flag'] = np.where(
#     df['ontime'] == 0, # The dataset seems to have no '0' values for 'ontime'
#     'NOT_ON_TIME_DELIVERY',
#     None
# )
# print(f"--- Number of 'NOT_ON_TIME_DELIVERY' flags detected: {df['not_ontime_delivery_anomaly_flag'].count()} ---")
# print("--- NOTE: 'not_ontime_delivery_anomaly_flag' is currently disabled as 'ontime' column contains only '1'. ---")



Anomaly Area 2: Product Rating Anomalies (Customer Feedback) ---

In [21]:
# Assuming 'product_rating' ranges from 1 to 5. Let's flag anything below a 2.5 rating as low.
# You can check df['product_rating'].describe() again to fine-tune this.
low_product_rating_threshold = 2.5 # Flag if rating is less than 2.5
df['low_product_rating_anomaly_flag'] = np.where(
    df['product_rating'] < low_product_rating_threshold,
    'LOW_PRODUCT_RATING',
    None
)
print(f"--- Number of 'LOW_PRODUCT_RATING' flags detected: {df['low_product_rating_anomaly_flag'].count()} ---")


--- Number of 'LOW_PRODUCT_RATING' flags detected: 202450 ---


In [23]:
# Display a sample of the DataFrame with new flags (selected columns)
print("\n--- DataFrame Head with New Anomaly Flags (selected columns) ---")
print(df[['order_date', 'estimated_delivery_time_days', 'long_estimated_delivery_anomaly_flag',
          'product_rating', 'low_product_rating_anomaly_flag'
         ]].head(10))


--- DataFrame Head with New Anomaly Flags (selected columns) ---
  order_date  estimated_delivery_time_days  \
0 2022-06-02                           3.0   
1 2022-06-11                           1.0   
2 2022-06-11                           4.0   
3 2022-06-11                           1.0   
4 2022-06-11                           5.0   
5 2022-06-11                           1.0   
6 2022-06-11                           4.0   
7 2022-06-11                           4.0   
8 2022-06-11                           3.0   
9 2022-06-11                           1.0   

  long_estimated_delivery_anomaly_flag  product_rating  \
0                                 None             3.0   
1                                 None             4.0   
2                                 None             5.0   
3                                 None             3.0   
4              LONG_ESTIMATED_DELIVERY             3.0   
5                                 None             3.0   
6                    

In [24]:
# Define the output file name for the cleaned data
output_file_name = 'e_commerce_shipping_data_cleaned.csv'

# Save the cleaned and transformed DataFrame to a new CSV file
# index=False prevents pandas from writing the DataFrame index (row numbers) as a column in the CSV.
df.to_csv(output_file_name, index=False)

print(f"\n--- Cleaned and transformed data saved successfully to {output_file_name} ---")
print("\n--- Final Data Info (after all transformations) ---")
df.info() # Displays the final state of your DataFrame


--- Cleaned and transformed data saved successfully to e_commerce_shipping_data_cleaned.csv ---

--- Final Data Info (after all transformations) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 507111 entries, 0 to 507110
Data columns (total 12 columns):
 #   Column                                Non-Null Count   Dtype         
---  ------                                --------------   -----         
 0   order_id                              507111 non-null  int64         
 1   product_id                            507111 non-null  int64         
 2   order_date                            507111 non-null  datetime64[ns]
 3   courier_delivery                      507110 non-null  object        
 4   city                                  507110 non-null  object        
 5   district                              507110 non-null  object        
 6   type_of_delivery                      507110 non-null  object        
 7   estimated_delivery_time_days          507111 non-null  floa